In [6]:
from google.cloud import storage

from google.oauth2 import service_account

SCOPES = ['https://www.googleapis.com/auth/drive.readonly','https://www.googleapis.com/auth/devstorage.full_control','https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = './credentials/fiery-protocol-399500-f2566dd92ef4.json'

creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
    # bucket_name = "your-bucket-name"

    storage_client = storage.Client(credentials=creds)

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name)

    # Note: The call returns a response only when the iterator is consumed.
    for blob in blobs:
        print(blob.name)

In [9]:
list_blobs('data-lake-henry')

business.pkl
metadata_11.json
metadata_3.json
metadata_4.json
metadata_5.json
metadata_9.json
review.json
tip.json
user.parquet


In [129]:
import numpy as np
import pandas as pd

df = pd.read_json('data/Google Maps/estados/review-Texas/1.json', lines=True)
df.user_id = df.user_id.astype(str)
df.user_id = df.user_id.apply(lambda x: x.replace('e+20','').replace('.',''))
df.user_id = df.user_id.apply(lambda x: int(x))
df.name = df.name.astype(str)
df.text = df.text.astype(str)
df.pics = df.pics.astype(str)
df.gmap_id = df.gmap_id.astype(str)
df.dtypes

df['resp_time'] = pd.Series()
df['resp_text'] = pd.Series()
for i in range(len(df)):
    # print(i)
    if type(df.resp[i]) == dict:
        df.loc[i,'resp_time'] = df.resp[i]['time']
        df.loc[i,'resp_text'] = df.resp[i]['text']
    else:
        df.loc[i,'resp_time'] = 1
        df.loc[i,'resp_text'] = ''
df.resp_time = df.resp_time.fillna(0).apply(lambda x: int(x))
df.resp_text = df.resp_text.fillna('')
df = df[['user_id', 'name', 'time', 'rating', 'text', 'gmap_id', 'resp_time', 'resp_text']]

ndjson = df.to_json(orient='records',lines=True) # 'data/Google Maps/estados/review-Texas/1_test.json',
with open('data/Google Maps/estados/review-Texas/1_test.ndjson','w') as f:
    f.write(ndjson)